# Pénzügyi adatelemzés 
### Első házi feladat
### Homoszkedaszticitás tesztelés

A házi feladatot itt a notebook-ban kell megoldani
- legfeljebb 4 fős csapatokban és
- ipynb (Jupyter Notebook) formátumban a Moodle-re feltölteni <span style="color:red">**2025-10-31 péntek du. 4**</span> óráig, csapatonként EGY példányban (tehát NE töltse fel mindenki).
- A csapatokra vonatkozólag nincs más megkötés, mint, hogy legfeljebb 4 fősek lehetnek, lehet dolgozni akár párban, akár egyedül is és nem szükséges a csapattagoknak ugyanabba a szemináriumi csoportba járni. 
- A csapattagok nevét, Neptun-kódját és "uni-corvinus"-os e-mail címét írjátok be a notebook második cellájába. Minden feltöltött file-ra az a hallgató fog pontot kapni, akinek az adatai itt hibátlanul szerepelnek. Ha ugyanaz a csapat több file-t is feltölt, akkor pontot vonunk le. 

Minden kérdés után van a szöveges válasznak vagy kódnak egy üres cella, de természetesen lehet újabb cellákat is beszúrni és további függvényeket írni.

Nem feltétlenül elegendő az előre megírt függvényeket kiegészíteni, a kódnak minden, a feladatben kért output-ot el kell készítenie és a cellákra sorban kattintva hibátlanul lefutnia. A feladat értékelésénél nem fogunk semmilyen további kódot hozzáírni, ha valaki elfelejt meghívni egy függvényt, akkor pontot veszíthet. Mivel a notebook egyetlen session, emlékszik minden változóra, akkor is, ha az azt létrehozó kódot később kitöröljük, így ellenőrzésként célszerű a végső verzió tesztelése előtt újraindítani a kernel-t vagy bezárni a file-t újraindítani a szervert, és úgy kipróbálni.

Az elméleti kérdésekre adott válaszok legyenek rövidek, pontosak, lényegre törőek, a szükséges mennyiségű, de nem több indoklással. Aki válaszként mindent is leír, (ahogy a chatGPT szokta), azt negatívan értékeljük. Az ábrákat nem kell "szép"-re formázni, megfelelnek az alapértelmezett beállítások. A kódok a kért feladatokat oldják meg, ne valami mást vagy általánosabbat, a számítások egyes lépéseit szükség esetén egy-két szavas kommentekkel el lehet látni.

AI indokolt esetben és módon használható, de meg kell jelölni, hogy hol, mire és miért használtátok. Ha Python kód generálásra használtok generatív AI-t, akkor másoljátok ki a promt-ot (az AI-nak adott utasítást) egy külön cellába.

A szimulációk paramétereit a beadott file-ban úgy állítsátok be, hogy a kódok futása gyors (legfeljebb néhány másodperc) legyen, de a szöveges válaszokban összefoglalhatjátok a szimulációk eredményeit nagyobb mintaelemszámra is. Sokáig futó kódhoz célszerű a `tqdm` package `tqdm` függvényét használni, ami egy szépen formázott progressbart készít egy `for` ciklushoz. 

**Csapat tagjai (név, neptun kód, e-mail-cím):**




In [1]:
# A cellákban külön importokat ne tegyünk, egy helyen töltsük be az összes felhasznált package-t. 
# Az összes feladat megoldható ezekkel (de persze lehet bővíteni a listát):

from matplotlib import pyplot as plt
import numpy as np
import statsmodels.api as sm
import seaborn as sns
from scipy import stats
from statsmodels.stats.diagnostic import het_breuschpagan
from tqdm import tqdm

#### 1. Feladat

Legyen $X$ és $Y$ független standard normális eloszlású valószínűségi változók, és legyen $m\in\mathbb{N}$ egy pozitív egész szám. Szimulációval becsüljük meg az 

\begin{equation}
Y^2\sim \beta_x X+\beta_0
\tag{SQ}
\end{equation}

regressziós egyenlet egy $m$ elemű mintából becsült bétájának és $R^2$-ének eloszlását. 

- A tanult elmélet alapján mit tudunk mondani $\beta_x$ és az $R^2$ eloszlásáról?
- Ábrázoljuk vizuálisan (hisztogram, sűrűségfüggvény) a szimulációval becsült $\beta_x$ és $R^2$ eloszlását.


<div style="color: blue">
Válasz:
</div>

<Axes: ylabel='Density'>

In [4]:
def sim_sqr(m):
    x = np.random.normal(size = m)
    X = sm.add_constant(x)
    y = np.random.normal(size = m)
    y2 = y**2

    res = sm.OLS(y2, X).fit()

    return res

R2 = [sim_sqr(30).rsquared for _ in range(1000)]
Beta = [sim_sqr(30).rsquared for _ in range(1000)]
sns.kdeplot(R2)
sns.kdeplot(Beta, color = "red")

#### 2. Feladat

Legyen $X$ standard normális eloszlású valószínűségi változó, és legyen 
\begin{equation}
Y = 2X + 1 + N\left(0, 0.04 + \alpha X^2\right)
\tag{H}
\end{equation}
ahol $\alpha\in\mathbb{R}$ egy pozitív paraméter, N a normális eloszlást jelöli, a paraméterek a várható érték és a variancia (a szimulációhoz ebből gyököt kell vonni, mert a numpy a normális eloszlást a várható értékkel és a szórással paraméterezi).

- Az $\alpha$ paraméter mely értéke(i) mellett tesz eleget a folyamat a homoszkedaszticitás feltételének és miért?
- Írjunk függvényt, ami adott $m$ és $\alpha$ mellett $m$ elemű mintát generál az $(X,Y)$ eloszlásból.

<div style="color: blue">
Válasz:
</div>

In [20]:
def generate_data(m, alpha):
    x = np.random.normal(size = m)
    y = 2*x+1+np.random.normal(0,(0.04+alpha*x**2)**0.5,size = m)
    return x, y

X, Y = generate_data(1000, 1)
plt.scatter(X, Y, s = 3)

#### 3. Feladat

Írjunk függvényt, ami szimulálja a $(H)$ formulával meghatározott eloszlású változókra illesztett regresszió egy $m$ elemű mintából becsült  együtthatóinak eloszlását.

- Miben és mennyire térnek el az eloszlások az $\alpha$ paramétertől függően?
- Torzítatlan, illetve
- konzisztens lesz-e a becslés?

<div style="color: blue">
Válasz:
</div>

In [ ]:
def sim_heteroscedasticity(m, alpha):
    return

#### 4. Feladat

A [Breusch-Pagan](https://en.wikipedia.org/wiki/Breusch%E2%80%93Pagan_test) teszt egy tetszőleges
$$ Y = \beta^Tx + \beta_0 + \varepsilon(\sigma_x)$$
lineáris regresszióban a homoszkedaszticitás detektálására szolgáló statisztikai teszt. Nullhipotézise, hogy $\sigma_x = \sigma$ az $x$-től valójában nem függő konstans. Technikailag úgy működik, hogy 

- elvégezzük a regresszió praméterbecslését,
- kiszámoljuk az $\varepsilon$ becsült hibatagokat és
- a variancia ML-becslését $\hat\sigma^2 = \frac{\sum\varepsilon_i}{n}$ (ez egy értelmetlen kifejezés, ha a nullhipotézis nem tejlesül),   
- majd ezekből kiszámoljuk a standardizált reziduálisok négyzetét: $g = \varepsilon^2/\hat\sigma^2$
- és ezeket újra regresszáljuk az eredeti magyarázó változókkal\begin{equation}g\sim\gamma^Tx + \gamma_0.\tag{S}\end{equation}

Megmutatható, hogy $$nR^2\stackrel{H_0}{\longrightarrow}\chi^2_{p-1},$$ ahol $R^2$ az (S) segéd-regresszió $R^2$-e, így erre tudunk egy aszimptotikus $\chi^2$-tesztet végezni.

írjunk egy saját függvényt, a `statsmodels.stats.diagnostic` package `het_breuschpagan` függvényének reprodukálására, azaz

- implementáljuk a Breusch-Pagan teszt segédregresszióját,
- számítsuk ki az $R^2$-et és a $\chi^2$-teszt $p$-értékét, és

az eredményt hasonlítsuk összes a beépített függvény output-jával. 

In [ ]:
def bp_test_with_built_in_function(data):
    return

In [ ]:
def my_bp_test(data):
    return

In [ ]:
# Összehasonlítás:

#### 5. Feladat

Szimuláljuk a BP-teszt segédregressziójában az együtthatók és az $R^2$ eloszlását a ($H$) formulával definiált változókara és hasonlítsuk össze az 1. Feladat-ban az ($SQ$) formulával meghatározott eloszlásra kapott eredményekkel. Hasonló eredményt kapunk-e amikor $\alpha=0$? Ha igen, miért, ha nem, miért nem?

<div style="color: blue">
Válasz:
</div>

In [ ]:
def bp_regression(data):
    return 

#### 6. Feladat

A korábban megírt függvények felhasználásával írjunk egy új függvényt, ami a $p$-érték helyett konkrét True/False értéket ad vissza a homoszkedaszticitás, mint nullhipotézis eldöntésére a Breusch-Pagan teszt alapján.

In [ ]:
def my_bp_homoscedasticity_test(data, sig_level = 0.95):
    return

Futtassunk szimulációt, amely az $\alpha$ paraméter különféle értékeire megvizsgálja, hogy mekkora valószínűséggel utasítja el BP-teszt a homoszkedaszticitást 95%-os szignifikanciaszinten. Milyen eredményekre számítunk az $\alpha$ függvényében, elméletileg? Ezeket az eredményeket kaptuk-e? Fogalmazzuk meg pár mondatban a konklúziókat.

In [ ]:
def bp_test_sim(alpha, sim_size = 1000):
    return

In [ ]:
# EREDMÉNYEK MEGJELENÍTÉSE

<div style="color: blue">
Válasz:
</div>



